In [1]:
%load_ext autoreload
%autoreload 2

# mount drive
from google.colab import drive
drive.mount('/content/drive')

# cd into project directory
%cd /content/drive/My\ Drive/Georgia_Tech/Spring_2021/sbic_stereotypes

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1M873oJOlIb1Bd5vliq6d04b61XsHLT5h/sbic_stereotypes


In [2]:
!pip install transformers
!pip install datasets

import torch
import pandas as pd
import numpy as np

     |████████████████████████████████| 1.9MB 19.1MB/s 
     |████████████████████████████████| 3.2MB 49.9MB/s 
     |████████████████████████████████| 890kB 48.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0b86a92acc970f9368b63035b4f2dad29f3e954b333e24468c0da55d824517f6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 184kB 22.0MB/s 
     |████████████████████████████████| 112kB 49.4MB/s 
     |████████████████████████████████| 245kB 39.6MB/s 


In [3]:
## Set all parameters here ##
gpt_dict_5epoch = {
                    'MODEL_NAME':'openai-gpt', \
                    'OUTPUT_DIR':'model/gpt_5epoch', \
                    'LEARNING_RATE': 5e-6, \
                    'SAVE_STEPS': 22367, \
                    'EPOCHS': 5.0
                  }

gpt2_dict_5epoch =  {
                      'MODEL_NAME':'gpt2', \
                      'OUTPUT_DIR':'model/gpt2_5epoch', \
                      'LEARNING_RATE': 1e-5, \
                      'SAVE_STEPS': 24075, \
                      'EPOCHS': 5.0
                    }

active_dict = gpt_dict_5epoch

In [4]:
from data_preprocessing import *

# Read and Clean Data
from_train_file = 'data/SBIC.v2.trn.csv'
from_dev_file = 'data/SBIC.v2.dev.csv'
to_train_file = 'data/train_text.csv'
to_dev_file = 'data/dev_text.csv'

clean_df(from_train_file, to_train_file)
clean_df(from_dev_file, to_dev_file)

In [5]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files={"train": to_train_file, "validation": to_dev_file})
print(datasets)

Using custom data configuration default-6f5834b455468d91


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6f5834b455468d91/0.0.0/965b6429be0fc05f975b608ce64e1fa941cc8fb4f30629b523d2390f3c0e1a93. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 112900
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 16738
    })
})


In [6]:
# We need to create the model and tokenizer
tokenizer = setup_tokenizer(active_dict['MODEL_NAME'])
tokenized_datasets = datasets.map(lambda examples: tokenizer(examples["text"]), \
                                    batched=True, num_proc=4, \
                                    remove_columns=["text"])

print(tokenized_datasets)





DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 112900
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 16738
    })
})


In [7]:
# Normalize the text length as Blocks
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    if total_length % block_size:
      remainder = block_size - (total_length % block_size)
    else:
      remainder = 0
    
    pad_extension = [tokenizer.pad_token_id for _ in range(remainder)]
    attention_extension = [0 for _ in range(remainder)]

    concatenated_examples['input_ids'].extend(pad_extension)
    concatenated_examples['attention_mask'].extend(attention_extension)
    total_length += remainder
    
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

print(lm_datasets)





DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 35785
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 5352
    })
})


In [8]:
# Load PreTrained Model and train it
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments

model = AutoModelForCausalLM.from_pretrained(active_dict['MODEL_NAME'])
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir = active_dict['OUTPUT_DIR'],
    evaluation_strategy = 'steps',
    eval_steps = 2000,
    save_steps = active_dict['SAVE_STEPS'],
    save_total_limit = 1,
    warmup_steps = 5000,
    learning_rate = active_dict['LEARNING_RATE'],
    per_device_train_batch_size = 4,
    num_train_epochs = active_dict['EPOCHS'],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

trainer.train()

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Runtime,Samples Per Second
2000,3.172600,2.907126,60.312200,88.738000
4000,2.646300,2.358180,60.402500,88.606000
6000,2.277700,2.017895,60.336000,88.703000
8000,2.081900,1.861661,60.264700,88.808000
10000,1.967800,1.796489,60.225500,88.866000
12000,1.921900,1.761221,60.324900,88.720000
14000,1.882000,1.733225,60.391400,88.622000
16000,1.831400,1.715443,60.203000,88.899000
18000,1.830000,1.702361,60.112000,89.034000
20000,1.787400,1.694591,59.962500,89.256000


Step,Training Loss,Validation Loss,Runtime,Samples Per Second
2000,3.172600,2.907126,60.312200,88.738000
4000,2.646300,2.358180,60.402500,88.606000
6000,2.277700,2.017895,60.336000,88.703000
8000,2.081900,1.861661,60.264700,88.808000
10000,1.967800,1.796489,60.225500,88.866000
12000,1.921900,1.761221,60.324900,88.720000
14000,1.882000,1.733225,60.391400,88.622000
16000,1.831400,1.715443,60.203000,88.899000
18000,1.830000,1.702361,60.112000,89.034000
20000,1.787400,1.694591,59.962500,89.256000


TrainOutput(global_step=44735, training_loss=1.967050637854562, metrics={'train_runtime': 9208.8876, 'train_samples_per_second': 4.858, 'total_flos': 16015034902118400, 'epoch': 5.0})